<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_4/01_simple_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなファインチューニング
最小限のコードでファインチューニングを実装します。  
事前学習済みのモデルに、何回か追加で訓練を行います。

## ライブラリのインストール
ライブラリTransformersをインストールします。  
https://huggingface.co/transformers/index.html

In [ ]:
!pip install transformers

## モデルの読み込み
`BertForSequenceClassification`により、事前学習済みのモデルを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification

sc_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", return_dict=True)
print(sc_model.state_dict().keys())

## 最適化アルゴリズム
今回は、最適化アルゴリズムに`AdamW`を採用します。  
`AdamW`は、オリジナルのAdamの重みの減衰に関する式を変更したものです。  
https://arxiv.org/abs/1711.05101

In [ ]:
from transformers import AdamW

optimizer = AdamW(sc_model.parameters(), lr=1e-5)

## Tokenizerの設定
`BertTokenizer`により文章を単語に分割し、idに変換します。  
`BertForSequenceClassification`のモデルの訓練時には入力の他にAttention maskを渡す必要があるのですが、`BertTokenizer`によりこちらも得ることができます。

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentences = ["I love baseball.", "I hate baseball."]
tokenized = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
print(tokenized)

x = tokenized["input_ids"]
attention_mask = tokenized["attention_mask"]

## ファインチューニング
事前に学習済みのモデルに対して、追加で訓練を行います。

In [ ]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

sc_model.train()
t = torch.tensor([1,0])  # 文章の分類

weight_record = []  # 重みを記録

for i in range(100):
    y = sc_model(x, attention_mask=attention_mask)
    loss = F.cross_entropy(y.logits, t)
    loss.backward()
    optimizer.step()

    weight = sc_model.state_dict()["bert.encoder.layer.11.output.dense.weight"][0][0].item()
    weight_record.append(weight)

plt.plot(range(len(weight_record)), weight_record)
plt.show()

追加の訓練により、重みが調整されていく様子が確認できます。